# Reading and processing data

In [1]:
import pandas as pd

dataframe = pd.read_csv("titanic.csv") # https://www.kaggle.com/datasets/heptapod/titanic

# Drop all columns that start with "zero"
dataframe = dataframe.loc[:, ~dataframe.columns.str.startswith('zero')]

# Rename '2urvived' to 'Survived'
dataframe = dataframe.rename(columns={"2urvived": "Survived"})

dataframe['Embarked'] = dataframe['Embarked'].fillna(0)

In [2]:
import numpy as np
import cupy as cp # Rapids libraby

colId = "Passengerid"
target = "Survived"

features = dataframe.drop([colId, target], axis=1).columns.tolist()

X = dataframe[features]
y = dataframe[target]

# Model 2NN (2 Layers, input-output)

In [6]:
class model_2NN:
    """
    Implementa uma Rede Neural Rasa para problemas de CLASSIFICAÇÃO BINÁRIA utilizando a função de ativação tangente hiperbólica nos neurônios de camada oculta e sigmóid para o neurônio da camada de saída.
    - Obtem primeiramente os tamanhos para criar todas as matriz e utilizar de vetorização. 
    - Inicializa os parâmetros de W de forma aleatória multiplicado por 0.01 e o viés com uma matriz de zeros
    - Itera várias vezes para realizar a propagação para a frente e para trás afim de atualizar os pesos para melhor se ajustar.
    - Realiza previões de novas instâncias para conseguir tanto as probabilidades de uma intância ter valor 1, quanto preve se ela tem o valor 1 ou 0.
    """
    def __init__(
        self, 
        iterations=1000, 
        learning_rate=0.01, 
        random_state=42, 
        hidden_layer_units=16, 
        output_size=1, 
        verbose=100,
        device='CPU',
    ):
        self.random_state = random_state
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.n_h = hidden_layer_units
        self.verbose = verbose
        self.device = device.lower()
        self.parameters = {}
        self.grads = {}
        self.cost = None

    def sigmoid(self, Z):
        """
        Calcula a sigmoide de Z
    
        Argumentos:
        Z -- Um array escalar ou numpy de qualquer tamanho.
    
        Retorno:
        s -- sigmoid(Z)
        """

        if self.device == 'cpu':
            Z_c=np.clip(Z,-500,500)
            s = 1 / (1 + np.exp(np.array(-Z_c)))
        else:
            Z_c=cp.clip(Z,-500,500)
            s = 1 / (1 + cp.exp(cp.array(-Z_c)))
        
        return s

    def layer_sizes(self, X, Y):
        """
        Argumentos:
        X -- conjunto de dados de entrada da forma (tamanho da entrada, número de exemplos)
        Y -- rótulos da forma (tamanho da saída, número de exemplos)
        
        Retorna:
        n_x -- o tamanho da camada de entrada
        n_h -- o tamanho da camada oculta
        n_y -- o tamanho da camada de saída
        """
        
        n_x = X.shape[0]
        n_h = self.n_h
        n_y = Y.shape[0]
        
        return n_x, n_h, n_y
    
    def initialize_parameters(self, n_x, n_h, n_y):
        """
        Argumento:
        n_x -- tamanho da camada de entrada
        n_h -- tamanho da camada oculta
        n_y -- tamanho da camada de saída
        
        Retorna:
        params -- dicionário Python contendo seus parâmetros:
        W1 -- matriz de pesos da forma (n_h, n_x)
        b1 -- vetor de viés da forma (n_h, 1)
        W2 -- matriz de pesos da forma (n_y, n_h)
        b2 -- vetor de viés da forma (n_y, 1)
        """    
        if self.device == 'cpu':
            W1 = np.random.randn(n_h, n_x) * 0.01
            b1 = np.zeros((n_h, 1))
            W2 = np.random.randn(n_y, n_h) * 0.01
            b2 = np.zeros((n_y, 1))
        else:
            W1 = cp.random.randn(n_h, n_x) * 0.01
            b1 = cp.zeros((n_h, 1))
            W2 = cp.random.randn(n_y, n_h) * 0.01
            b2 = cp.zeros((n_y, 1))
    
        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2}
        
        return parameters

    def forward_propagation(self, X, parameters):
        """
        Argumento:
        X -- dado de entrada de tamanho (n_x, m)
        parameters -- dicionário Python contendo seus parâmetros (saída da função de inicialização)
        
        Retorna:
        A2 -- A saída sigmoide da segunda ativação
        cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2"
        """
        
        W1 = parameters['W1']
        b1 = parameters['b1']
        W2 = parameters['W2']
        b2 = parameters['b2']

        if self.device == 'cpu':
            Z1 = np.dot(W1, X) + b1
            A1 = np.tanh(Z1)
            Z2 = np.dot(W2, A1) + b2
        else:
            Z1 = cp.dot(W1, X) + b1
            A1 = cp.tanh(Z1)
            Z2 = cp.dot(W2, A1) + b2
        
        A2 = self.sigmoid(Z2)

        assert(A2.shape == (1, X.shape[1]))
        
        cache = {"Z1": Z1,
                 "A1": A1,
                 "Z2": Z2,
                 "A2": A2}
        
        return A2, cache
        
    def compute_cost(self, A2, Y):
        """
        Calcula o custo de entropia cruzada (logloss)
        
        Argumentos:
        A2 -- A saída sigmoide da segunda ativação, da forma (1, número de exemplos)
        Y -- vetor de rótulos "verdadeiro" da forma (1, número de exemplos)
        
        Retorna:
        custo -- custo de entropia cruzada
        
        """
        
        m = Y.shape[1] # number of examples

        if self.device == 'cpu':
            cost = - (1 / m) * np.sum((Y * np.log(A2) + (1 - Y) * np.log(1 - A2)))
            cost = float(np.squeeze(cost))
        else:
            cost = - (1 / m) * cp.sum((Y * cp.log(A2) + (1 - Y) * cp.log(1 - A2)))
            cost = float(cp.squeeze(cost))
        return cost

    def backward_propagation(self, parameters, cache, X, Y):
        """
        Implementa a propagação reversa.
        
        Argumentos:
        parameters -- dicionário Python contendo os parâmetros
        cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2".
        X -- dados de entrada da forma (n_x, m)
        Y -- rótulos "true" do vetor da forma (1, m)
        
        Retorna:
        grads -- dicionário Python contendo seus gradientes em relação a diferentes parâmetros
        """
        m = X.shape[1]
        
        W1 = parameters['W1']
        W2 = parameters['W2']
        
        A1 = cache['A1']
        A2 = cache['A2']
        Z1 = cache['Z1']
        
        dZ2 = A2 - Y
        if self.device == 'cpu':
            dW2 = (1 / m) * np.dot(dZ2, A1.T)
            db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
            dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
            dW1 = (1 / m) * np.dot(dZ1, X.T)
            db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
        else:
            dW2 = (1 / m) * cp.dot(dZ2, A1.T)
            db2 = (1 / m) * cp.sum(dZ2, axis=1, keepdims=True)
            dZ1 = cp.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
            dW1 = (1 / m) * cp.dot(dZ1, X.T)
            db1 = (1 / m) * cp.sum(dZ1, axis=1, keepdims=True)
        
        grads = {"dW1": dW1,
                 "db1": db1,
                 "dW2": dW2,
                 "db2": db2}
        
        return grads

    def update_parameters(self, parameters, grads, learning_rate):
        """
        Atualiza parâmetros usando a regra de atualização de gradiente descendente fornecida acima.
        
        Argumentos:
        parameters -- dicionário Python contendo seus parâmetros
        grads -- dicionário Python contendo seus gradientes
        learning_rate -- taxa de aprendizado 
        
        Retorna:
        parameters -- dicionário Python contendo seus parâmetros atualizados
        """
        
        W1 = parameters['W1']
        W2 = parameters['W2']
        b1 = parameters['b1']
        b2 = parameters['b2']
        
        dW1 = grads['dW1']
        db1 = grads['db1']
        dW2 = grads['dW2']
        db2 = grads['db2']
        
        W1 = W1 - (learning_rate * dW1)
        b1 = b1 - (learning_rate * db1)
        W2 = W2 - (learning_rate * dW2)
        b2 = b2 - (learning_rate * db2)
        
        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2}
        
        return parameters

    def set_data_type(self, X, Y=None):
        np.random.seed(self.random_state)
        if self.device == 'cpu':
            X = np.array(X.T)
            if Y is not None: Y = np.array(Y).reshape(1, -1)
        else:
            cp.random.seed(self.random_state)
            X = cp.array(X.T)
            if Y is not None: Y = cp.array(Y).reshape(1, -1)
        return X, Y

    def fit(self, X, Y):
        """
        Argumentos:
        X -- conjunto de dados da forma (n_x, m)
        Y -- rótulos da forma (1, m)
        num_iterations -- Número de iterações no loop de descida de gradiente
        
        Retorna:
        parameters -- parâmetros aprendidos pelo modelo. Eles podem então ser usados ​​para prever.
        """
        
        X, Y = self.set_data_type(X, Y)
            
        n_x, n_h, n_y = self.layer_sizes(X, Y)
        self.n_x = n_x
        
        parameters = self.initialize_parameters(n_x, n_h, n_y)
    
        for i in range(0, self.iterations):
            
            A2, cache = self.forward_propagation(X, parameters)
            
            cost = self.compute_cost(A2, Y)
            
            grads = self.backward_propagation(parameters, cache, X, Y)
            
            parameters = self.update_parameters(parameters, grads, self.learning_rate)
            
            if self.verbose > 0 and i % self.verbose == 0:
                print ("Cost after iteration %i: %f" %(i, cost))
    
        self.parameters = parameters
        
        return self

    def predict_proba(self, X):
        """
        Usando os parametros aprendidos, prever as classes para cada exemplo em X
        
        Argumentos:
        X -- entrada dos dados de size (n_x, m)
        
        Retorno
        previsões -- vetor de probabilidades preditas do modelo 
        """
        
        X, _ = self.set_data_type(X)

        A2, _ = self.forward_propagation(X, self.parameters)

        if self.device == 'gpu': A2 = A2.get()

        return A2

    def predict(self, X):
        """
        Usando os parametros aprendidos, prever as classes para cada exemplo em X
        
        Argumentos:
        X -- entrada dos dados de size (n_x, m)
        
        Retorno
        previsões -- vetor de previsões do modelo 0 ou 1
        """
        
        probabilities = self.predict_proba(X)
        threshold = 0.5
        predictions = (probabilities > threshold).astype(int)
                
        return predictions.reshape(-1,)
    

# Modeling - Baseline

### CPU

In [4]:
%%time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler

ini_params_2nn = {
    'iterations':200,
    'learning_rate':0.05, 
    'random_state':42,
    'hidden_layer_units':1024,
    'verbose':-100,
}

oof_predictions = np.zeros(len(X), dtype=int)

NUM_FOLDS = 6
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    scaler = StandardScaler()
    
    Xtrn_fold, Xtst_fold = scaler.fit_transform(X.iloc[trn_idx]), scaler.transform(X.iloc[val_idx])
    ytrn_fold, ytst_fold = y.iloc[trn_idx], y.iloc[val_idx]

    #print(f"##### FOLD {fold} #####")
    model_instance = model_2NN(
        **ini_params_2nn,
        device='CPU',
    )

    model_instance.fit(Xtrn_fold, ytrn_fold)
    predictions = model_instance.predict(Xtst_fold)
    
    oof_predictions[val_idx] = predictions
    
    f1_fold = f1_score(ytst_fold, predictions)
    print(f"F1-score [fold-{fold}]: {f1_fold}")

f1 = f1_score(y, oof_predictions)

print(f"\nF1-score: {f1:.5f}\n\n")
print(f"\n{classification_report(y, oof_predictions)}")

F1-score [fold-0]: 0.5116279069767442
F1-score [fold-1]: 0.41975308641975306
F1-score [fold-2]: 0.45454545454545453
F1-score [fold-3]: 0.3950617283950617
F1-score [fold-4]: 0.38202247191011235
F1-score [fold-5]: 0.4186046511627907

F1-score: 0.43053



              precision    recall  f1-score   support

           0       0.80      0.94      0.86       967
           1       0.65      0.32      0.43       342

    accuracy                           0.78      1309
   macro avg       0.72      0.63      0.65      1309
weighted avg       0.76      0.78      0.75      1309

CPU times: user 1min 35s, sys: 3min 31s, total: 5min 7s
Wall time: 53 s


### GPU

In [5]:
%%time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler

oof_predictions = np.zeros(len(X), dtype=int)

NUM_FOLDS = 6
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    scaler = StandardScaler()
    
    Xtrn_fold, Xtst_fold = scaler.fit_transform(X.iloc[trn_idx]), scaler.transform(X.iloc[val_idx])
    ytrn_fold, ytst_fold = y.iloc[trn_idx], y.iloc[val_idx]

    #print(f"##### FOLD {fold} #####")
    model_instance = model_2NN(
        **ini_params_2nn,
        device='GPU',
    )

    model_instance.fit(Xtrn_fold, ytrn_fold)
    predictions = model_instance.predict(Xtst_fold)
    
    oof_predictions[val_idx] = predictions
    
    f1_fold = f1_score(ytst_fold, predictions)
    print(f"F1-score [fold-{fold}]: {f1_fold}")

f1 = f1_score(y, oof_predictions)

print(f"\nF1-score: {f1:.5f}\n\n")
print(f"\n{classification_report(y, oof_predictions)}")

F1-score [fold-0]: 0.47619047619047616
F1-score [fold-1]: 0.41975308641975306
F1-score [fold-2]: 0.45454545454545453
F1-score [fold-3]: 0.3950617283950617
F1-score [fold-4]: 0.38202247191011235
F1-score [fold-5]: 0.4186046511627907

F1-score: 0.42436



              precision    recall  f1-score   support

           0       0.80      0.94      0.86       967
           1       0.65      0.32      0.42       342

    accuracy                           0.78      1309
   macro avg       0.72      0.63      0.64      1309
weighted avg       0.76      0.78      0.75      1309

CPU times: user 1.99 s, sys: 734 ms, total: 2.73 s
Wall time: 952 ms


# Optimize hyperparams with device GPU

In [7]:
import optuna
NUM_FOLDS = 6
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

def objective(trial):

    iterations=trial.suggest_int('iterations', 50, 4000, step=50)
    learning_rate=trial.suggest_float('learning_rate', 0.00001, 0.01, log=True) 
    hidden_layer_units=2 ** (trial.suggest_int('hidden_layer_units', 2, 12))

    oof_predictions = np.zeros(len(X), dtype=int)

    NUM_FOLDS = 6
    skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    
    for fold, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
        scaler = StandardScaler()
        
        Xtrn_fold, Xtst_fold = scaler.fit_transform(X.iloc[trn_idx]), scaler.transform(X.iloc[val_idx])
        ytrn_fold, ytst_fold = y.iloc[trn_idx], y.iloc[val_idx]
    
        #print(f"##### FOLD {fold} #####")
        model_instance = model_2NN(
            iterations=iterations,
            learning_rate=learning_rate, 
            hidden_layer_units=hidden_layer_units,
            random_state=42,
            verbose=-100,
            device='GPU'
        )
    
        model_instance.fit(Xtrn_fold, ytrn_fold)
        predictions = model_instance.predict(Xtst_fold)
        
        oof_predictions[val_idx] = predictions
        
        f1_fold = f1_score(ytst_fold, predictions)
        print(f"F1-score [fold-{fold}]: {f1_fold}")
    
    f1 = f1_score(y, oof_predictions)

    return f1

study = optuna.create_study(direction='maximize') # Queremos maximizar o F1 score

N_TRIALS = 50 # Número de conjuntos de hiperparâmetros a testar (ajuste conforme tempo/recursos)
study.optimize(objective, n_trials=N_TRIALS)

[I 2025-04-30 16:33:58,219] A new study created in memory with name: no-name-34a5f9a4-1504-45b9-aa6a-03d0118cd108


F1-score [fold-0]: 0.0
F1-score [fold-1]: 0.0
F1-score [fold-2]: 0.0
F1-score [fold-3]: 0.0
F1-score [fold-4]: 0.0


[I 2025-04-30 16:34:01,150] Trial 0 finished with value: 0.0 and parameters: {'iterations': 250, 'learning_rate': 0.003933520591271461, 'hidden_layer_units': 7}. Best is trial 0 with value: 0.0.


F1-score [fold-5]: 0.0
F1-score [fold-0]: 0.0
F1-score [fold-1]: 0.0
F1-score [fold-2]: 0.0
F1-score [fold-3]: 0.0
F1-score [fold-4]: 0.0


[I 2025-04-30 16:34:28,071] Trial 1 finished with value: 0.0 and parameters: {'iterations': 2750, 'learning_rate': 0.005854915529892832, 'hidden_layer_units': 2}. Best is trial 0 with value: 0.0.


...

F1-score [fold-5]: 0.0
F1-score [fold-0]: 0.3611111111111111
F1-score [fold-1]: 0.5234899328859061
F1-score [fold-2]: 0.4503311258278146
F1-score [fold-3]: 0.5147058823529411
F1-score [fold-4]: 0.4720496894409938


[I 2025-04-30 16:47:29,298] Trial 48 finished with value: 0.48709315375982043 and parameters: {'iterations': 900, 'learning_rate': 1.4689305028590635e-05, 'hidden_layer_units': 12}. Best is trial 31 with value: 0.5348189415041783.


F1-score [fold-5]: 0.6
F1-score [fold-0]: 0.0
F1-score [fold-1]: 0.0
F1-score [fold-2]: 0.0
F1-score [fold-3]: 0.0
F1-score [fold-4]: 0.0


[I 2025-04-30 16:47:35,500] Trial 49 finished with value: 0.0 and parameters: {'iterations': 600, 'learning_rate': 0.0037860766588490562, 'hidden_layer_units': 6}. Best is trial 31 with value: 0.5348189415041783.


F1-score [fold-5]: 0.0


# Final model 

In [8]:
final_params_2nn = study.best_params
final_params_2nn

{'iterations': 1300,
 'learning_rate': 2.5705490185275697e-05,
 'hidden_layer_units': 12}

In [9]:
final_params_2nn['hidden_layer_units'] = 2 ** final_params_2nn['hidden_layer_units']
final_params_2nn['verbose'] = -1
final_params_2nn['device'] = 'GPU'
final_params_2nn['random_state'] = 42

final_params_2nn

{'iterations': 1300,
 'learning_rate': 2.5705490185275697e-05,
 'hidden_layer_units': 4096,
 'verbose': -1,
 'device': 'GPU',
 'random_state': 42}

In [10]:
%%time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler


oof_predictions = np.zeros(len(X), dtype=int)

NUM_FOLDS = 6
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    scaler = StandardScaler()
    
    Xtrn_fold, Xtst_fold = scaler.fit_transform(X.iloc[trn_idx]), scaler.transform(X.iloc[val_idx])
    ytrn_fold, ytst_fold = y.iloc[trn_idx], y.iloc[val_idx]

    #print(f"##### FOLD {fold} #####")
    model_instance = model_2NN(
        **final_params_2nn,
    )

    model_instance.fit(Xtrn_fold, ytrn_fold)
    predictions = model_instance.predict(Xtst_fold)
    
    oof_predictions[val_idx] = predictions
    
    f1_fold = f1_score(ytst_fold, predictions)
    print(f"F1-score [fold-{fold}]: {f1_fold}")

f1 = f1_score(y, oof_predictions)

print(f"\nF1-score: {f1:.5f}\n\n")
print(f"\n{classification_report(y, oof_predictions)}")

F1-score [fold-0]: 0.4036697247706422
F1-score [fold-1]: 0.5245901639344263
F1-score [fold-2]: 0.5254237288135594
F1-score [fold-3]: 0.5892857142857143
F1-score [fold-4]: 0.5401459854014599
F1-score [fold-5]: 0.6166666666666667

F1-score: 0.53482



              precision    recall  f1-score   support

           0       0.84      0.81      0.82       967
           1       0.51      0.56      0.53       342

    accuracy                           0.74      1309
   macro avg       0.67      0.69      0.68      1309
weighted avg       0.75      0.74      0.75      1309

CPU times: user 22.1 s, sys: 2.23 s, total: 24.3 s
Wall time: 26.1 s
